# Imports

In [7]:
try:
    from requests_html import HTMLSession
except: 
    !pip install requests_html
    from requests_html import HTMLSession
    
try:
    import pandas as pd
    import numpy as np
except:
    !pip install pandas
    import pandas as pd
    import numpy as np

# # try:
# #     from Stockie.stockie import stockie
# # except:
# #     !pip install stockie
# #     !pip install seaborn
# #     from Stockie.stockie import stockie

# # try:
# #     import yfinance as yf
# # except:
# #     !pip install yfinance
# #     import yfinance as yf

# try:
#     from mplfinance.original_flavor import candlestick_ohlc
# except:
#     !pip install mplfinance
#     from mplfinance.original_flavor import candlestick_ohlc
    
try:
    import plotly.graph_objects as go
    import plotly.subplots as ms
except:
    !pip install plotly
    import plotly.graph_objects as go
    import plotly.subplots as ms
    
try:
    from datetime import datetime
except:
    !pip install datetime
    from datetime import datetime
    
try:
    import time
except:
    !pip install time
    import time
    
try:
    import requests
except:
    !pip install requests
    import requests

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
import os.path

import warnings
warnings.filterwarnings('ignore')

# Functions

In [8]:
def market_info(market):
    market = market.replace('(', '')
    market = market.replace(')', '')
    market = market.replace(' ', '-')

    url = 'https://www.ingsprinters.nl/sprinters/' + market

    r = HTMLSession().get(url)

    value = r.html.find('span', clean=True)
    
    return float(value[4].text.strip().replace('.', '').replace(',', '.'))

In [9]:
def csv_log_file(ticker, df):
    if os.path.isfile('../data/' + ticker + '_log.csv'):
        df.to_csv('../data/' + ticker + '_log.csv',
                       sep=';',
                       decimal=',',
                       index=False)
    else:
        df.to_csv('../data/' + ticker + '_log.csv',
                  sep=';',
                  decimal=',',
                  index=False)

In [10]:
def INGSprinter_candlestick(ticker, refreshtime, interval, file_mgmt,
                            candlestick,candlestick_timer):
    i = 0
    if file_mgmt and os.path.isfile('../data/' + ticker + '_log.csv'):
        df = pd.read_csv('../data/' + ticker + '_log.csv',
                         sep=';',
                         decimal=',')
        df = df[df.Time >= datetime.today().strftime('%Y-%m-%d')]
    else:
        df = pd.DataFrame(columns=['Time', 'Rate'])
    while i < refreshtime / interval:
        a = datetime.now()
        #Extract rate and time
        df = df.append({
            'Time': datetime.now(),
            'Rate': market_info(ticker)
        },
                       ignore_index=True)
        print(market_info(ticker), datetime.now())
        b = datetime.now()
        
        if candlestick:
            df_copy=df.copy()
            df_copy['Time'] = pd.to_datetime(df_copy['Time'])
            df_copy=df_copy.set_index('Time')
            df_new = df_copy['Rate'].resample(candlestick_timer).ohlc()
        else:
            df.plot(x='Time', y=['Rate'], figsize=(16, 9))
            plt.show()
        
        fig = go.Figure(data=[
            go.Candlestick(x=df_new.index,
                           open=df_new['open'],
                           high=df_new['high'],
                           low=df_new['low'],
                           close=df_new['close'])
        ])
        fig.show()
        time.sleep(interval - (b - a).total_seconds())
        clear_output(wait=True)
        i = i + 1

    if file_mgmt:
        csv_log_file(ticker, df)

# Settings

In [13]:
interval=10
refreshtime=8*60*60
ticker='DAX'
candlestick=True
candlestick_timer='15Min'
if get_ipython(
).__class__.__name__ == 'ZMQInteractiveShell':  #Check if the code is running in Jupyter Notebook
    file_mgmt=True
else: 
    file_mgmt=False

# Run!

In [14]:
INGSprinter_candlestick(ticker,refreshtime ,interval,file_mgmt,candlestick,candlestick_timer)

15850.18 2021-08-11 20:56:10.412702


In [ ]:
#Next step: Candlestick pattern detection!